In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import codecs, json
import unicodedata
# pip install Unidecode  <OR> conda install Unidecode
import unidecode

In [2]:
#importing all csv files
domarar = pd.read_csv('csv/blak-domarar.csv', sep=';', header=0)
einstaklingar = pd.read_csv('csv/blak-einstaklingar.csv', sep=';', header=0)
forsvarsmenn = pd.read_csv('csv/blak-forsvarsmenn.csv', sep=';', header=0)
lid = pd.read_csv('csv/blak-lid.csv', sep=';', header=0)
lidimoti = pd.read_csv('csv/blak-lidimoti.csv', sep=';', header=0)
lidsmenn = pd.read_csv('csv/blak-lidsmenn.csv', sep=';', header=0)
lidsstjorar = pd.read_csv('csv/blak-lidsstjorar.csv', sep=';', header=0)
thjalfarar = pd.read_csv('csv/blak-thjalfarar.csv', sep=';', header=0)
mot = pd.read_csv('csv/blak-mot.csv', sep=';', header=0)

# drop all SyndarLids with an ID (SyndarlidID)
# (the reason for not dropping using SyndarLid is because I don't trust that column to be inserted correctly with [0,1])
lid = lid[lid['SyndarlidID'].isna()]
# then dropping those two columns because we don't want virtual teams
lid = lid.drop(columns=['SyndarLid', 'SyndarlidID'])

# All duplicated birthdays
duplicated_einstaklingar = einstaklingar[einstaklingar.duplicated(subset=['Nafn', 'Fdagur', 'Kyn'], keep=False)]
duplicated_fdagur_kyn_einstaklingar = einstaklingar[einstaklingar.duplicated(subset=['Fdagur', 'Kyn'], keep=False)]


In [3]:
# Add all entries that have duplicated birthdays, then filter that to first_name->birthday-><people entries>
duplicate_dict = defaultdict(dict)
for index, row in duplicated_fdagur_kyn_einstaklingar.iterrows():
    full_name = row['Nafn']
    #only get the first part of full name 
    first_name = full_name.split()[0]
    # make first name lowercase
    first_name_lowercase = first_name.lower()
    # encode icelandic letters to english
    first_name_to_english = unidecode.unidecode(first_name_lowercase)
    # split birthday into year month and day and ignore second part (sec, min, hour)
    Fdagur_date = row['Fdagur'].split()[0]
    
    if first_name_to_english in duplicate_dict.keys():
        if Fdagur_date in duplicate_dict[first_name_to_english].keys():
            #if first name and Fdagur (birthday) exist in dict then append to that key (birthday)
            duplicate_dict[first_name_to_english][Fdagur_date].append(row.values)
        else:
            #if first name exists but Fdagur (birthday) does not exist in dict
            duplicate_dict[first_name_to_english][Fdagur_date] = [row.values]
    else:
        #if Fdagur (birthday) does not exist in dict
        duplicate_dict[first_name_to_english][Fdagur_date] = [row.values]
        

In [4]:
# Remove all single birthday entries that are not duplicates
dict_removed_single_entries = defaultdict(dict)

for key, values in duplicate_dict.items():
    # key = nafn ('ludvik')
    for birthday, arrays in dict(values).items():
        # only get duplicates that there exists 2 or more entries for a birthday
        if(len(arrays) > 1):
            # used for when joining teams table
            if key in dict_removed_single_entries.keys():
                if birthday in dict_removed_single_entries[key].keys():
                    dict_removed_single_entries[key][birthday].append(arrays)
                else:
                    #if first name exists but Fdagur (birthday) does not exist in dict
                    dict_removed_single_entries[key][birthday] = arrays
            else:
                dict_removed_single_entries[key][birthday] = arrays

In [5]:
# get all ids in dict_removed_single_entries
duplicate_ids_kept = []
for key, values in dict_removed_single_entries.items():
    # key = nafn ('ludvik')
    for birthday, arrays in dict(values).items():
        for item in arrays:
            duplicate_ids_kept.append(item[0])

In [6]:
#duplicate_dict['aldis']
len(duplicate_ids_kept)

1329

In [7]:
#dict_removed_single_entries.values()
#duplicate_row_keept

In [8]:
# Checking if two names are the same person
dict_duplicate_compare_team_members = defaultdict(dict)
for index, row in lidsmenn.iterrows():
    ids = row["EinstID"]
    if ids in duplicate_ids_kept:
        # now we only view ids that exist for duplicated people
        #print(ids)
        if ids in dict_duplicate_compare_team_members.keys():
            dict_duplicate_compare_team_members[ids].append(row.values)
        else:
            dict_duplicate_compare_team_members[ids] = [row.values]


In [9]:
#dict_duplicate_compare_team_members

In [10]:
dict_name_entries = {}
for key, value in dict_removed_single_entries.items():
    #get key and arrays for each person
    for birthday, arrays in dict(value).items():
        #get each array for person
        #print("KEY: " + key + " BIRTHDAY: " + birthday)
        new_key = key +"-"+ birthday
        for item in arrays:
            if new_key in dict_name_entries.keys():
                dict_name_entries[new_key].append(item[0])
            else:
                dict_name_entries[new_key] = [item[0]]
#dict_name_entries

In [11]:
dict_duplicate_compare_team_members[1964]

[array([187, 2840, 1964, '2011-01-20 21:19:21.280'], dtype=object),
 array([232, 1022, 1964, '2013-09-03 20:55:14.783'], dtype=object),
 array([234, 1022, 1964, '2013-09-09 15:35:13.947'], dtype=object),
 array([249, 2840, 1964, '2014-04-21 21:12:11.077'], dtype=object)]

In [14]:
dict_einstaklingar_teammember_info = {}
for key, value in dict_name_entries.items():
    #print("<key>" + str(key) + " <value> " + str(value))
    for item in value:
        #print(item)
        if item in dict_duplicate_compare_team_members.keys():
            #print("<key>" + str(key) + " <item> " + str(item))
            for compare_arrays in dict_duplicate_compare_team_members[item]:
                mot_id = compare_arrays[0]
                lid_id = compare_arrays[1]
                player_id = compare_arrays[2]
                date_played = compare_arrays[3].split()[0]
                
                temp = (str(player_id) + " " + str(lid_id) + " " + str(compare_arrays[3]))   
                if key in dict_einstaklingar_teammember_info.keys():
                    dict_einstaklingar_teammember_info[key].append(temp)
                else:
                    dict_einstaklingar_teammember_info[key] = [temp]
        #print("xxxxxxxxxxxx")

In [13]:
dict_einstaklingar_teammember_info

{'adalsteinn-1981-03-03': ['1964 2840 2011-01-20 21:19:21.280',
  '1964 1022 2013-09-03 20:55:14.783',
  '1964 1022 2013-09-09 15:35:13.947',
  '1964 2840 2014-04-21 21:12:11.077',
  '1437 1023 2004-07-26 12:25:47.497',
  '1437 1023 2003-10-11 19:12:00.843',
  '1437 1023 2004-09-17 11:59:19.497',
  '1437 1023 2004-09-30 17:47:04.420',
  '1437 1015 2004-11-15 23:01:41.150',
  '1437 2085 2005-02-23 18:40:23.027',
  '1437 1023 2005-10-01 20:06:49.467',
  '1437 1022 2010-09-27 10:28:20.920',
  '1437 1022 2011-08-29 20:53:45.547',
  '1437 2840 2012-03-04 14:10:07.770',
  '1437 2690 2012-04-04 20:14:56.530',
  '1437 5601 2012-01-13 00:28:42.140',
  '1437 1022 2012-07-06 10:43:22.870',
  '1437 2840 2013-03-27 19:40:58.193',
  '1437 1022 2012-10-09 10:49:09.633',
  '1437 2840 2013-03-27 19:42:14.360',
  '1437 2840 2013-03-13 21:09:03.820',
  '1437 1830 2014-08-14 10:15:41.083'],
 'aldis-1996-06-22': ['2949 4378 2010-02-25 13:52:40.360',
  '2950 4378 2010-02-25 13:52:42.780'],
 'alexander-1990-

In [ ]:
#dict(duplicated_fdagur_kyn_einstaklingar)

# all duplicates
#duplicate_dict

# all duplicates for lúðvík
#duplicate_dict["ludvik"]

#duplicate_dict["ludvik"]['1969-03-31'][0]
#duplicate_dict["ludvik"]['1969-03-31'][1]

#duplicate_dict
#dict_removed_single_entries

In [ ]:
reverted_back_to_dict = dict(duplicate_dict)
#reverted_back_to_dict

In [ ]:
file_path = "json/einstaklingar_map.txt" ## your path variable
#duplicate_dict_json = json.dump(duplicate_dict, codecs.open(file_path, 'w', encoding='utf-8'), separators=(';', ':'), sort_keys=True, indent=4) ### this saves the array in .json format
#json_obj = json.dumps(duplicate_dict, indent = 4)
#dumped = json.dumps(duplicate_dict, cls=NumpyEncoder)
#dumped
#pd.DataFrame(reverted_back_to_dict).to_csv(file_path, encoding='utf-8-sig')
#duplicate_dict_json = json.dump(reverted_back_to_dict, codecs.open(file_path, 'w', encoding='utf-8-sig'))

#json = json.dumps(reverted_back_to_dict)
#f = open(file_path,"w")
#f.write(str(reverted_back_to_dict))
#f.close()

=====================================================================================
=

In [ ]:
#FINAL STEP (run after everything is done):

#duplicated people put into it's own csv to be browsed later
pd.DataFrame(duplicated_einstaklingar).to_csv("csv/new/duplicated-einstaklingar.csv", encoding='utf-8-sig')
pd.DataFrame(duplicate_dict).to_csv("json/duplicate-map.json", encoding='utf-8-sig')


#save as new csv inside csv/new
pd.DataFrame(domarar).to_csv("csv/new/blak-domarar.csv", encoding='utf-8-sig')
pd.DataFrame(einstaklingar).to_csv("csv/new/blak-einstaklingar.csv", encoding='utf-8-sig')
pd.DataFrame(forsvarsmenn).to_csv("csv/new/blak-forsvarsmenn.csv.csv", encoding='utf-8-sig')
pd.DataFrame(lid).to_csv("csv/new/blak-lid.csv", encoding='utf-8-sig')
pd.DataFrame(lidimoti).to_csv("csv/new/blak-lidimoti.csv", encoding='utf-8-sig')
pd.DataFrame(lidsmenn).to_csv("csv/new/blak-lidsmenn.csv", encoding='utf-8-sig')
pd.DataFrame(lidsstjorar).to_csv("csv/new/blak-lidsstjorar.csv", encoding='utf-8-sig')
pd.DataFrame(mot).to_csv("csv/new/blak-mot.csv", encoding='utf-8-sig')
pd.DataFrame(thjalfarar).to_csv("csv/new/blak-thjalfarar.csv", encoding='utf-8-sig')